In [56]:
import json
from bs4 import BeautifulSoup
from markdownify import markdownify
from dotenv import find_dotenv, load_dotenv
import os
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS

In [52]:
# Load environment variables
load_dotenv(find_dotenv())

True

## Step 1: Create content for vector db

In [13]:
def html_to_markdown(html_string):
    # Parse the HTML string using BeautifulSoup
    soup = BeautifulSoup(html_string, 'html.parser')

    # Convert the parsed HTML to Markdown using markdownify
    markdown_text = markdownify(str(soup))

    return markdown_text

In [3]:
with open('./../data/qa-data.json', 'r') as file:
    data = json.load(file)

In [91]:
data[0]

{'id': '62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
 'type': 'vraag en antwoord',
 'canonical': 'https://www.rijksoverheid.nl/onderwerpen/leerplicht/vraag-en-antwoord/leerplicht-schoolvakanties',
 'dataurl': 'https://opendata.rijksoverheid.nl/v1/infotypes/faq/62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
 'question': 'Mag ik mijn kind meenemen op vakantie buiten de schoolvakantie?',
 'lastmodified': '2023-05-31T13:52:55.841Z',
 'answer': {'id': '62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
  'type': 'vraag en antwoord',
  'canonical': 'https://www.rijksoverheid.nl/onderwerpen/leerplicht/vraag-en-antwoord/leerplicht-schoolvakanties',
  'dataurl': 'https://opendata.rijksoverheid.nl/v1/infotypes/faq/62bdc113-aee1-4469-bc36-4c7c3fc39a4d',
  'question': 'Mag ik mijn kind meenemen op vakantie buiten de schoolvakantie?',
  'introduction': '<p>U mag uw kind niet meenemen op vakantie buiten de schoolvakanties. U kunt een boete krijgen als u dit wel doet. Als u in de schoolvakanties niet weg kunt door uw werk kun

In [93]:
def create_content_snippet(question):
    # Get all content pieces in the answer
    if "content" in question["answer"].keys():
        content_pieces = list(
            map(
                lambda content_piece: [html_to_markdown(paragraph) for paragraph in content_piece.values()],
                question["answer"]["content"])
            )
        concat_content = "\n".join([item for row in content_pieces for item in row])
    else:
        concat_content = ""

    return f"""Vraag: {question["question"]}
Antwoord:
{html_to_markdown(question["answer"]["introduction"]) if "introduction" in question["answer"].keys() else ""}
{concat_content}

{"Onderwerpen: " + ", ".join(question["answer"]["subjects"]) if "subjects" in question["answer"].keys() else ""}
{"Thema's: " + ", ".join(question["answer"]["themes"]) if "themes" in question["answer"].keys() else ""}
{"Autoriteit: " + question["answer"]["authority"] if "authority" in question["answer"].keys() else ""}"""

In [98]:
for qa in data:
    qa["content"] = create_content_snippet(qa)

/var/folders/4n/btbbzjrn31nbw6c4jkw2yr1c0000gn/T/ipykernel_52874/3634077436.py:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_string, 'html.parser')
/Users/martijnschouten/Library/Caches/pypoetry/virtualenvs/feedbackfrontiers-3xN2_Ao--py3.11/lib/python3.11/site-packages/markdownify/__init__.py:96: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


## Step 2: Build vector db

In [101]:
# Ready all models
embedding = OpenAIEmbeddings()
llm = ChatOpenAI(model_name="gpt-4", temperature=0)

In [102]:
vectordb_persist_dir = "./../data/faiss_index"

In [104]:
if not os.path.exists(vectordb_persist_dir):
    factors = [
        Document(
            page_content=qa_item["content"],
            metadata={
                "identifier": qa_item["id"],
                "source": qa_item["canonical"]
            },
        )
        for qa_item in data
    ]

    # Create vector store
    vectordb = FAISS.from_documents(
        documents=factors,
        embedding=embedding,
    )
    vectordb.save_local(vectordb_persist_dir)
else:
    # ChromaDB has been initialised before, recreate instance
    vectordb = FAISS.load_local(vectordb_persist_dir, embedding)

In [124]:
def get_answer_from_llm(query):    
    # Build prompt
    template = """Gedraag je als een helpvolle assistent voor mensen die op zoek zijn naar allerlei antwoorden op vragen die iets te maken hebben met de Rijksoverheid. Beantwoord deze vraag ALLEEN op basis van de gegeven bronnen, niet op basis van eigen kennis. Als je de vraag niet kan beantwoorden, verontschuldig je en zeg dat de webmaster op de hoogte is gebracht van het niet hebben van de gevraagde informatie.
    Bronnen: ```{context}```
    Vraag: ```{question}```
    Behulpzaam antwoord: """
    qa_chain_prompt = PromptTemplate.from_template(template)

    # Define search kwargs
    search_kwargs = {"k": 5}

    # Create QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(search_kwargs=search_kwargs),
        return_source_documents=True,
        chain_type_kwargs={"prompt": qa_chain_prompt},
    )

    # Get result
    result = qa_chain({"query": query})

    return {
        **result,
        "source_documents": [
            {
                "page_content": doc.page_content,
                "identifier": doc.metadata["identifier"],
                "source": doc.metadata["source"],
            }
            for doc in result["source_documents"]
        ]
    }

In [125]:
get_answer_from_llm("Zit er BTW op zonnepanelen?")

{'query': 'Zit er BTW op zonnepanelen?',
 'result': 'Sinds 2023 is de btw op zonnepanelen 0%. Dit nultarief geldt alleen als de zonnepanelen worden geplaatst op woningen of bijgebouwen van een woning.',
 'source_documents': [{'page_content': "Vraag: Krijg ik subsidie voor zonnepanelen?\nAntwoord:\nEr zijn verschillende financiële regelingen om de aankoop en het gebruik van zonnepanelen aantrekkelijk te maken.\xa0\n\n\nGoedkoop geld lenen voor aankoop en installatie van zonnepanelen\nOm zonnepanelen te betalen, kunt u gebruikmaken van de [Energiebespaarlening](https://www.warmtefonds.nl/particulieren/energiebesparende-maatregelen/zonnepanelen). Met deze lening kunt u energiebesparende maatregelen in of aan uw huis betalen.\n\n\nGeen btw op zonnepanelen\n------------------------\n\n\n Sinds 2023 is de btw op zonnepanelen 0%. Dit nultarief geldt alleen als de zonnepanelen worden geplaatst op woningen of bijgebouwen van een woning.\n\n\nRegeling voor zelfgeproduceerde elektriciteit (salder